## How to use the Data Lab query manager service

This notebook documents how to query the Data Lab via the query manager service. This can be done either from a Python script or from the command line using the <i>datalab</i> command.

### The query manager service interface

The query manager service simplifies access to the Data Lab databases. This section describes the query manager service interface in case we want to write our own code against that rather than using one of the provided tools.
The query manager service accepts an HTTP GET call to the <i>query</i> endpoint with the following parameters:

| Name | Function | Optional | Supported values |
|------|----------|----------|------------------|
| adql | The query string to run against the db | No | - |
| out | The location to save any results | Yes | 'vos://...', 'mydb://...' |
| ofmt | The output format of any results | Yes | ascii, csv, votable |
| async | Run the query asynchronously | Yes | true/false |
| profile | The server profile to use | Yes | default |

For example: /query?adql=<query>&ofmt=csv

#### Saving results
If no save location is specified (no <i>out</i> param) then the results are returned directly. A save location beginning with the 'vos://' identifier indicates a location in the user's virtual storage to save the result. A save location beginning with the 'mydb://' identifier indicates the results are to be saved to a table in the user's remote database (MyDB). 

#### Output formats
The results can be returned as whitespace delimited (<i>ascii</i>), CSV (<i>csv</i>), or in VOTable format (<i>votable</i>). Note that if the results are saved to the user's database then the output format is ignored.

#### Asynchronous queries
Long queries should be run asynchronously and the service may refuse any synchronous query attempted if the projected query time is too long. A query can be submitted asynchronously by setting the <i>async</i> parameter to <i>true</i>. A job id will then be returned.

The status of an asynchronous query can be checked by submitting an HTTP GET call to the query manager service <i>status</i> endpoint with the relevant job id as an argument: /status?jobid=<jobid>. A return value of 'COMPLETED' indicates the query has terminated. A return value of 'ERROR' indicates that there was a problem with the query.

The results of an asynchronous query (assuming that they were not saved to either the user's virtual storage or remote database) can be retrieved once the query has completed with an HTTP GET call to the query manager service <i>results</i> endpoint with the relevant job id as argument: /results?jobid=<jobid>

#### Authentication
The query manager service requires a Data Lab security token. This needs to be passed as the value of the header keyword "X-DL-AuthToken" in any HTTP GET call to the service. 

#### Profiles
There are several different backend machines serving data, etc., and it is possible that a user might want to specify a particular set to run their query on and save results to. A <i>profile</i> defines such a set and a particular profile can be specified in a query call.

### From Python code

The query manager service can be called from Python code using the <i>datalab</i> module. This provides methods to access the various query manager functions in the <i>queryMgr</i> subpackage. 

#### Initialization
This is the setup that is required to use the query manager. The first thing to do is import the relevant Python modules and also retrieve our DataLab security token (remember that this has to be included in any call to the query manager service).

In [2]:
from dl import auth
from dl import queryClient 

# Get the security token for the datalab user
token = auth.login('dldemo', 'dldemo')

#### The queryClient class

All queries are executed through the <i>query</i> method of the <i>queryClient</i> class. This takes as arguments:

| Argument | Description | Default  value |
|----------|-------------|----------------|
| adql | The query to be submitted | None |
| fmt | The returned format (if any) | ascii |
| out | The saved location (if any) | None |
| async | Indicates if the query is asynchronous | False |
| profile | Indicates which backend machines to use | default |

Note that if a profile is not specified then the default profile is assumed.

#### A quick query

Let's say we want to return the $gri$ magnitudes of the top 100 objects in the SMASH data set and get it back as a CSV file:

In [3]:
query = 'select id, gmag, rmag, imag from smash.avgmag limit 100'
response = queryClient.query(token, adql = query, fmt = 'csv')

In [4]:
print response

id,gmag,rmag,imag
Hydra_60.10003,22.843399,24.6819,22.8244991
Hydra_60.89,24.8066998,24.2763996,24.0221004
Hydra_60.9953,23.9937,23.4176006,22.9069004
Hydra_60.10009,23.7912998,22.4797993,22.0340996
Hydra_60.9957,24.6268997,24.5195007,23.9092999
Hydra_60.9860,24.7607002,23.3785,24.0037994
Hydra_60.181,24.5746002,23.1380005,21.9060001
Hydra_60.103,24.1401005,24.3003006,23.0058002
Hydra_60.9922,25.2731991,24.8771,
Hydra_60.9863,23.2695007,24.4512997,23.8591003
Hydra_60.10022,23.7537003,23.6933994,23.6445999
Hydra_60.10043,24.8637009,24.7749996,23.9365997
Hydra_60.219,24.4347,23.6812,22.6504002
Hydra_60.216,23.9703007,24.2101002,23.1686993
Hydra_60.10081,24.3715,24.2938004,23.5991993
Hydra_60.10198,23.7346001,24.3570995,24.4517002
Hydra_60.360,23.8528004,24.0013008,23.7865009
Hydra_60.251,24.8265991,24.2684994,24.2320004
Hydra_60.304,25.1567001,24.2383003,23.9722004
Hydra_60.247,23.1860008,22.3076992,22.1240997
Hydra_60.9828,23.8581009,22.7644997,22.5168991
Hydra_60.351,24.6161003,24.0077

#### Saving results to virtual storage

Now we want to save the results from the same query to our virtual storage space instead.

In [134]:
response = queryClient.query(token, adql = query, fmt = 'csv', out = 'vos://mags.csv')

#### Saving results to remote database

Alternatively we may want to store the results in a table called <i>mags</i> in our remote database.

In [136]:
response = queryClient.query(token, adql = query, out = 'mydb://mags')

#### An asynchronous query

We now want to run a longer query, say, counting the total number of objects is USNO-B1, and need to do it asynchronously. The first thing we do is submit the query as normal but with the <i>async</i> argument indicated - this will return the id of the asynchronous job. All the previous arguments can also be used to specify where and in what format we want the query results.

In [154]:
query = 'select count(*) from usno.b1'
jobId = queryClient.query(token, adql = query, async = True)

ex70ziffiynv6t3c


We can check on the status of the job at any time:

In [157]:
status = queryClient.status(token, jobId = jobId)
print status

COMPLETED


If the status value is "QUEUED" then the job is waiting to be executed. If
it is "ERROR" then there was a problem with the execution. When the status value is "COMPLETED", we can get our results (assuming we did not save them to our virtual storage or remote database).

In [158]:
results = queryClient.results(token, jobId = jobId)
print results

COUNT
1045175762



#### Using profiles

Different data sets (or versions of the same data set) may reside on different backend database servers and a user may want to work explicitly with a particular set. The first thing to do is see what profiles are available.

We can get the details of a particular profile by including the name of the profile as an argument in the <i>list_profiles</i> method:

In [ ]:
queryClient.list_profiles(token, "default")

So let's try a query against the default profile first - let's get a list of all tables in the default database:

In [ ]:
adql = 'select table_catalog, table_schema, table_name from information_schema.tables'
default = queryClient.query(token, adql)

And now we'll run the same query against the 'zeus1' profile:

In [ ]:
queryClient.set_profile('zeus1')
zeus1 = queryClient.query(token, adql)

Comparing the two outputs, we can see that there are differences in which tables are available.

### Using the datalab command

The <i>datalab</i> command provides an alternate command line way to work with the query manager through the <i>query</i> subcommands.

#### Initialization
We need to be logged into the Data Lab to use the query manager.

In [ ]:
!datalab login --user=dldemo --password=dldemo

#### A quick query

Again we will return the magnitudes of the top 100 objects in the SMASH data set and get it back as a CSV file:

In [ ]:
!datalab query --ofmt='csv' \
  --adql='select id, gmag, rmag, imag from smash.avgmag limit 100'

#### Saving results to virtual storage

Now we'll save the results to our virtual storage:

In [1]:
!datalab query --out='vos://mags.csv' --ofmt='csv' \
    --adql='select id, gmag, rmag, imag from smash.avgmag limit 100'

/bin/sh: datalab: command not found


#### Saving results to remote database

And to our remote database:

In [2]:
!datalab query --out='mydb://mags' --ofmt='csv' \
    --adql='select id, gmag, rmag, imag from smash.avgmag limit 100' 

/bin/sh: datalab: command not found


#### An asynchronous query

Alternatively, we'll run the longer asynchronous query:

In [ ]:
!datalab query --adql='select count(*) from usno.b1' --async=True